In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
temp = pd.read_json('./datasets/case.json')
type(temp)

pandas.core.frame.DataFrame

In [3]:
temp.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [4]:
# transform to Brazilian timezone
temp["EnqueuedTimeUtc"] = pd.to_datetime(arg=temp["EnqueuedTimeUtc"]).dt.tz_convert(tz="Brazil/East")
temp.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [5]:
temp.EventName.value_counts()

DynamicPrice_Result    32
CurateOffer_Result      5
Name: EventName, dtype: int64

In [6]:
unique_eventnames = list(temp.EventName.unique())
unique_eventnames

['DynamicPrice_Result', 'CurateOffer_Result']

In [7]:
for i in unique_eventnames:
    if(i=='DynamicPrice_Result'):
        DynamicPrice_Result = temp[temp['EventName']==i]
        DynamicPrice_Result.reset_index(inplace=True)
    else:
        CurateOffer_Result = temp[temp['EventName']==i]
        CurateOffer_Result.reset_index(inplace=True)

DynamicPrice_Result.head()

,index,EnqueuedTimeUtc,EventName,Payload
0,0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [8]:
CurateOffer_Result.head()

,index,EnqueuedTimeUtc,EventName,Payload
0,3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
1,13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
2,15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
3,17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
4,35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [9]:
json.loads(DynamicPrice_Result.Payload[0])

{'provider': 'ApplyDynamicPriceRange',
 'offerId': 'a6611d55-9624-4381-8cdd-323ee3689241',
 'algorithmOutput': {'min_global': 85.0,
  'min_recommended': 87.2,
  'max_recommended': 97.65,
  'differenceMinRecommendMinTheory': 2.2}}

In [10]:
def dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result

lst = DynamicPrice_Result.apply(dynamic_price_option, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_option_payload = pd.DataFrame(flatten)
dynamic_price_option_payload

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/21
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/21
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/21
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21


In [29]:
def dynamic_price_range(row):
    EnqueuedTimeSP = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    result = []
    # payload_row
    if payload_row['provider'] == 'ApplyDynamicPriceRange':
        provider = payload_row['provider']
        offerId = payload_row['offerId']
        unique_option  = payload_row['algorithmOutput']
        price_range = {
                "provider" : provider,
                "offerId" : offerId,
                "MinGlobal" : unique_option['min_global'],
                "MinRecommended": unique_option['min_recommended'],
                "MaxRecommended": unique_option['max_recommended'],
                "DifferenceMinRecommendMinTheory": unique_option['differenceMinRecommendMinTheory'],
                "EnqueuedTimeSP": EnqueuedTimeSP
            }
        result.append(price_range)
    return result

lst = DynamicPrice_Result.apply(dynamic_price_range, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_range_payload = pd.DataFrame(flatten)
dynamic_price_range_payload



,provider,offerId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,ApplyDynamicPriceRange,a6611d55-9624-4381-8cdd-323ee3689241,85.00,87.20,97.65,2.2,05/09/21
1,ApplyDynamicPriceRange,b8c636fa-8241-47dc-ac40-bdf438a04d9c,85.00,87.20,97.65,2.2,05/09/21
2,ApplyDynamicPriceRange,3d32f7fb-396d-4d3f-b673-dea1f7dc41b7,85.00,87.20,97.65,2.2,05/09/21
3,ApplyDynamicPriceRange,329194f3-95a4-45ef-b3d0-2796f74ce2a0,85.00,87.20,97.65,2.2,05/09/21
4,ApplyDynamicPriceRange,fdcfde5c-113d-4a59-9ae0-8bc31e2943d8,87.35,89.25,99.95,1.9,05/09/21
5,ApplyDynamicPriceRange,27bbc4fa-2388-4780-b66c-92a51397d191,87.35,89.25,99.95,1.9,05/09/21
6,ApplyDynamicPriceRange,baffc30b-7642-45fe-a2ce-da31a71732ae,85.00,87.20,97.65,2.2,05/09/21
7,ApplyDynamicPriceRange,b5982abd-f602-47ac-b45a-bb43bf993d46,85.00,87.20,97.65,2.2,05/09/21
8,ApplyDynamicPriceRange,f6643886-4a0f-45ae-ae32-ee95c72ee94a,87.35,89.25,99.95,1.9,05/09/21
9,ApplyDynamicPriceRange,16a2d492-b1c3-40ec-970f-b8704d8db96f,85.00,87.20,97.65,2.2,05/09/21


In [43]:
def curated_offer(row):
    EnqueuedTimeSP = CurateOffer_Result['EnqueuedTimeUtc'][1].to_pydatetime().strftime("%d/%m/%y")
    payload_row = CurateOffer_Result['Payload'][1]
    payload_row = json.loads(payload_row)
    result = []
    for el in payload_row:
        CurationProvider = el['curationProvider']
        offerId = el['offerId']
        dealerId = el['dealerId']
        for unique_option in el['options']:
            curation_payload = {
                "CurationProvider":CurationProvider,
                "OfferId": offerId,
                "DealerId": dealerId,
                "UniqueOptionId": unique_option['uniqueOptionId'],
                "OptionId": unique_option['optionId'],
                "IsMobileDealer": unique_option['isMobileDealer'],
                "IsOpen": unique_option['isOpen'],
                "Eta": unique_option['eta'],
                "ChamaScore": unique_option['chamaScore'],
                "ProductBrand": unique_option['productBrand'],
                "IsWinner": unique_option['isWinner'],
                "MinimumPrice": unique_option['minimumPrice'],
                "MaximumPrice": unique_option['maximumPrice'],
                "DynamicPrice": unique_option['dynamicPrice'],
                "FinalPrice": unique_option['finalPrice']
            }
            if 'defeatPrimaryReason' in unique_option:
                curation_payload['DefeatPrimaryReason'] = unique_option['defeatPrimaryReason']
                curation_payload['DefeatReasons'] = unique_option['defeatReasons']
            else:
                curation_payload["DefeatPrimaryReason"] = "\"\""
                curation_payload["DefeatReasons"] = "\"\""
            curation_payload['EnqueuedTimeSP'] = EnqueuedTimeSP
        result.append(curation_payload)
    return result


flatten = [x for xs in CurateOffer_Result.apply(curated_offer, axis=1) for x in xs]
curated_offers_payload = pd.DataFrame(flatten)
curated_offers_payload

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,6517,d6562c24-0b37-5fb4-8275-65b7b8b47b87,6517 || 6517,False,True,0:01,8.0,ULTRAGAZ,False,90.0,180.0,91.90,91.90,HasDriverInOffer,[HasDriverInOffer],25/08/21
1,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,9047,f9b876ab-2590-952f-d69d-5b352ec251f3,9047 || 9047 || NACIONALGAS,False,False,1:00,9.0,NACIONALGAS,True,88.9,177.8,90.80,90.80,"""""","""""",25/08/21
2,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,8330,151e59ac-761a-96f5-d2b9-882037a9fd28,8330 || 8330 || CONSIGAZ,False,True,0:01,8.0,CONSIGAZ,True,80.0,160.0,87.35,87.35,"""""","""""",25/08/21
3,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,8835,b7a7b6d1-4dae-7392-5aaf-f3369c29db1d,8835 || 8835 || LIQUIGAS,False,False,0:01,8.0,LIQUIGAS,True,60.0,120.0,87.35,87.35,"""""","""""",25/08/21
4,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,6517,d6562c24-0b37-5fb4-8275-65b7b8b47b87,6517 || 6517,False,True,0:01,8.0,ULTRAGAZ,False,90.0,180.0,91.90,91.90,HasDriverInOffer,[HasDriverInOffer],25/08/21
5,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,9047,f9b876ab-2590-952f-d69d-5b352ec251f3,9047 || 9047 || NACIONALGAS,False,False,1:00,9.0,NACIONALGAS,True,88.9,177.8,90.80,90.80,"""""","""""",25/08/21
6,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,8330,151e59ac-761a-96f5-d2b9-882037a9fd28,8330 || 8330 || CONSIGAZ,False,True,0:01,8.0,CONSIGAZ,True,80.0,160.0,87.35,87.35,"""""","""""",25/08/21
7,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,8835,b7a7b6d1-4dae-7392-5aaf-f3369c29db1d,8835 || 8835 || LIQUIGAS,False,False,0:01,8.0,LIQUIGAS,True,60.0,120.0,87.35,87.35,"""""","""""",25/08/21
8,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,6517,d6562c24-0b37-5fb4-8275-65b7b8b47b87,6517 || 6517,False,True,0:01,8.0,ULTRAGAZ,False,90.0,180.0,91.90,91.90,HasDriverInOffer,[HasDriverInOffer],25/08/21
9,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,9047,f9b876ab-2590-952f-d69d-5b352ec251f3,9047 || 9047 || NACIONALGAS,False,False,1:00,9.0,NACIONALGAS,True,88.9,177.8,90.80,90.80,"""""","""""",25/08/21


In [52]:
sample_type = type(DynamicPrice_Result["Payload"][0])
print(f"The data type before using apply function: {sample_type}\n")
#column names that 
json_columns = ['Payload']

def clean_json(x):
    return json.loads(x)

for x in json_columns:
    DynamicPrice_Result[x] = DynamicPrice_Result[x].apply(clean_json)

sample_type2 = type(DynamicPrice_Result["Payload"][0])
print(f"The data type after using apply function: {sample_type2}")

The data type before using apply function: <class 'str'>

The data type after using apply function: <class 'dict'>


C:\Users\18572\AppData\Local\Temp\ipykernel_20492\3320935277.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DynamicPrice_Result[x] = DynamicPrice_Result[x].apply(clean_json)


In [83]:
sample_type3 = type(CurateOffer_Result["Payload"][3])
print(f"The data type before using apply function: {sample_type3}\n")
#column names that 
json_columns = ['Payload']

def clean_json(x):
    return json.loads(x)

for x in json_columns:
    CurateOffer_Result[x] = CurateOffer_Result[x].apply(clean_json)

sample_type4 = type(CurateOffer_Result["Payload"][3])
print(f"The data type after using apply function: {sample_type4}")

The data type before using apply function: <class 'str'>

The data type after using apply function: <class 'list'>


C:\Users\18572\AppData\Local\Temp\ipykernel_20492\1183088804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CurateOffer_Result[x] = CurateOffer_Result[x].apply(clean_json)


In [89]:
CurateOffer_Result.Payload[1]

[{'curationProvider': 'ByPrice',
  'offerId': 'c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d',
  'dealerId': '6517',
  'options': [{'uniqueOptionId': 'b0e296a9-0590-f0e0-8211-243a2ededb12',
    'optionId': '6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2',
    'isMobileDealer': True,
    'isOpen': True,
    'eta': '00:12:54.9215999',
    'chamaScore': 8.0,
    'productBrand': 'ULTRAGAZ',
    'isWinner': True,
    'minimumPrice': 90.0,
    'maximumPrice': 180.0,
    'dynamicPrice': 91.9,
    'finalPrice': 91.9},
   {'uniqueOptionId': 'd6562c24-0b37-5fb4-8275-65b7b8b47b87',
    'optionId': '6517 || 6517',
    'isMobileDealer': False,
    'isOpen': True,
    'eta': '0:01',
    'chamaScore': 8.0,
    'productBrand': 'ULTRAGAZ',
    'isWinner': False,
    'minimumPrice': 90.0,
    'maximumPrice': 180.0,
    'dynamicPrice': 91.9,
    'finalPrice': 91.9,
    'defeatPrimaryReason': 'HasDriverInOffer',
    'defeatReasons': ['HasDriverInOffer']}]},
 {'curationProvider': 'ByPrice',
  'offerId': 'c99aa9a7-ac5